In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

# READING THE CASE YAHOO TICKER SAMPLE LIST!
df = pd.read_excel("yahoo_ticker_sample_scraped.xlsx")
# LIST OF FIELDS WE WILL SCRAPE
list_of_fields = ['Market Cap', 'Enterprise Value', 'Trailing P/E', 'Forward P/E', 'PEG Ratio', 'Price/Sales', 'Price/Book', 'Enterprise Value/Revenue', 'Enterprise Value/EBITDA', 'Fiscal Year Ends', 'Most Recent Quarter', 'Profit Margin', 'Operating Margin', 'Return on Assets', 'Return on Equity', 'Revenue', 'Revenue Per Share', 'Quarterly Revenue Growth', 'Gross Profit', 'EBITDA', 'Net Income Avi to Common', 'Diluted EPS', 'Quarterly Earnings Growth', 'Total Cash', 'Total Cash Per Share', 'Total Debt', 'Total Debt/Equity', 'Current Ratio', 'Book Value Per Share', 'Operating Cash Flow', 'Levered Free Cash Flow', 'Beta', '52-Week Change', 'S&amp;P500 52-Week Change', '52 Week High', '52 Week Low', '50-Day Moving Average', '200-Day Moving Average', 'Avg Vol (3 month)', 'Avg Vol (10 day)', 'Shares Outstanding', 'Float', '% Held by Insiders', '% Held by Institutions', 'Shares Short', 'Short Ratio', 'Short % of Float', 'Shares Short (prior month)', 'Forward Annual Dividend Rate', 'Forward Annual Dividend Yield', 'Trailing Annual Dividend Rate', 'Trailing Annual Dividend Yield', '5 Year Average Dividend Yield', 'Payout Ratio', 'Dividend Date', 'Ex-Dividend Date', 'Last Split Factor', 'Last Split Date']

for fieldname in list_of_fields:
    df[fieldname]= df[fieldname].fillna('0')
    
list_of_fields_nodates = ['Market Cap', 'Enterprise Value', 'Trailing P/E', 'Forward P/E', 'PEG Ratio', 'Price/Sales', 'Price/Book', 'Enterprise Value/Revenue', 'Enterprise Value/EBITDA', 'Profit Margin', 'Operating Margin', 'Return on Assets', 'Return on Equity', 'Revenue', 'Revenue Per Share', 'Quarterly Revenue Growth', 'Gross Profit', 'EBITDA', 'Net Income Avi to Common', 'Diluted EPS', 'Quarterly Earnings Growth', 'Total Cash', 'Total Cash Per Share', 'Total Debt', 'Total Debt/Equity', 'Current Ratio', 'Book Value Per Share', 'Operating Cash Flow', 'Levered Free Cash Flow', '52-Week Change', 'S&amp;P500 52-Week Change', '52 Week High', '52 Week Low', '50-Day Moving Average', '200-Day Moving Average', 'Avg Vol (3 month)', 'Avg Vol (10 day)', 'Shares Outstanding', 'Float', '% Held by Insiders', '% Held by Institutions', 'Shares Short', 'Short Ratio', 'Short % of Float', 'Shares Short (prior month)', 'Forward Annual Dividend Rate', 'Forward Annual Dividend Yield', 'Trailing Annual Dividend Rate', 'Trailing Annual Dividend Yield', '5 Year Average Dividend Yield', 'Payout Ratio']


#TRANSFORMING TEXT (example 3.17B) INTO NUMBER INTO REAL NUMBERS (example 3,170,000,000).
d = {
     'k':  3,
     'M':  6,
     'B':  9,
     'T': 12,
     '%': -2
}
def text_to_num(text):
    try:
        if text[-1] in d:
            num, magnitude = text[:-1], text[-1]
            return float(num) * 10 ** d[magnitude]  #this case is when "text" has T, B, M, k or %
        else:
            return float(text) #this case is when "text" is string but look like a numeric
    except:
        try:
            return 1.0*text #this is when "text" is already numeric
        except:
            return 0.0 #it will reach this case when it is impossible to transform into numeric
        
#Example on how to call the function: text_to_num('3.17B')    
for fieldname in list_of_fields_nodates:
    df[fieldname] = np.vectorize(text_to_num)(df[fieldname])

IN_MODEL = []
for fieldname in list_of_fields_nodates:
    try:
        df[fieldname+'_group'] = df.groupby(['sector'])[fieldname].transform(
                             lambda x: pd.qcut(x, [0.0714, 0.2143, .5, 0.7857, 0.9286, 1.], labels=range(1,6)))
        df[fieldname+'_group']= 5 - df[fieldname+'_group'].fillna(0)
        IN_MODEL.append(fieldname+'_group')
    except:
        df[fieldname+'_group'] = 0
        continue

output_var = 'Beta'
X = df[list(IN_MODEL)]
y = df[output_var]

try:
    model = sm.OLS(y.astype(float),X.astype(float))
    result = model.fit()
    print (result.summary())
    y_pred = result.predict(X)
except np.linalg.linalg.LinAlgError as err:
    if 'Singular matrix' in err.message:
        print ("MODEL-INVALID (Singular Matrix)")
    else:
        raise

df['pred'] = y_pred

# SAVING TO EXCEL ALL PREDICTED  DATA!
df.to_excel("yahoo_ticker_sample_scraped_grouped_predicted.xlsx")

/home/chema/anaconda3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


                            OLS Regression Results                            
Dep. Variable:                   Beta   R-squared:                       0.932
Model:                            OLS   Adj. R-squared:                  0.873
Method:                 Least Squares   F-statistic:                     15.75
Date:                Thu, 06 Jun 2019   Prob (F-statistic):           1.78e-16
Time:                        22:17:59   Log-Likelihood:                -15.993
No. Observations:                  86   AIC:                             112.0
Df Residuals:                      46   BIC:                             210.2
Df Model:                          40                                         
Covariance Type:            nonrobust                                         
                                           coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------


In [2]:
print(df)

   ticker country      sector RefIndex    Market Cap  Enterprise Value  \
0    GOOG     USA  Technology      SPY  7.878800e+11      6.835700e+11   
1    MSFT     USA  Technology      SPY  6.956200e+11      6.465700e+11   
2      FB     USA  Technology      SPY  5.168600e+11      4.800800e+11   
3       T     USA  Technology      SPY  2.257600e+11      3.426400e+11   
4    ORCL     USA  Technology      SPY  2.070600e+11      1.940500e+11   
5      VZ     USA  Technology      SPY  2.100900e+11      3.247500e+11   
6     TSM     USA  Technology      SPY  2.135000e+11      2.034400e+11   
7    INTC     USA  Technology      SPY  2.077000e+11      2.165000e+11   
8    CSCO     USA  Technology      SPY  2.030600e+11      1.663800e+11   
9     IBM     USA  Technology      SPY  1.562700e+11      1.851600e+11   
10   NVDA     USA  Technology      SPY  1.351300e+11      1.308200e+11   
11    ACN     USA  Technology      SPY  9.866000e+10      9.508000e+10   
12    TXN     USA  Technology      SPY

In [3]:
# READING THE CASE YAHOO TICKER SAMPLE LIST!
df2 = pd.read_excel("yahoo_ticker_sample_scraped.xlsx")
print(df2)

   ticker country      sector RefIndex Market Cap Enterprise Value  \
0    GOOG     USA  Technology      SPY    787.88B          683.57B   
1    MSFT     USA  Technology      SPY    695.62B          646.57B   
2      FB     USA  Technology      SPY    516.86B          480.08B   
3       T     USA  Technology      SPY    225.76B          342.64B   
4    ORCL     USA  Technology      SPY    207.06B          194.05B   
5      VZ     USA  Technology      SPY    210.09B          324.75B   
6     TSM     USA  Technology      SPY     213.5B          203.44B   
7    INTC     USA  Technology      SPY     207.7B           216.5B   
8    CSCO     USA  Technology      SPY    203.06B          166.38B   
9     IBM     USA  Technology      SPY    156.27B          185.16B   
10   NVDA     USA  Technology      SPY    135.13B          130.82B   
11    ACN     USA  Technology      SPY     98.66B           95.08B   
12    TXN     USA  Technology      SPY    115.64B          111.23B   
13    VOD     USA  T

In [11]:
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(15,8))
fig = sm.graphics.plot_partregress_grid(result, fig=fig)

In [20]:
df['Rating_group'] = pd.qcut(df['pred'], [0, 0.0714, 0.2143, .5, 0.7857, 0.9286, 1.], labels=['D','CC','B','BB','BBB','A'] )

In [25]:
prueba = pd.qcut(df['pred'], [0, 0.0714, 0.2143, .5, 0.7857, 0.9286, 1.], labels=['D','CC','B','BB','BBB','A'] )

In [19]:
print (prueba)

0     1.0
1     2.0
2     0.0
3     NaN
4     4.0
5     NaN
6     0.0
7     1.0
8     2.0
9     3.0
10    3.0
11    2.0
12    2.0
13    1.0
14    3.0
15    1.0
16    3.0
17    2.0
18    NaN
19    2.0
20    1.0
21    1.0
22    2.0
23    2.0
24    2.0
25    4.0
26    0.0
27    1.0
28    2.0
29    1.0
     ... 
56    2.0
57    1.0
58    1.0
59    0.0
60    1.0
61    3.0
62    2.0
63    2.0
64    2.0
65    4.0
66    1.0
67    2.0
68    0.0
69    0.0
70    1.0
71    1.0
72    1.0
73    4.0
74    2.0
75    4.0
76    4.0
77    0.0
78    3.0
79    1.0
80    NaN
81    3.0
82    1.0
83    2.0
84    0.0
85    2.0
Name: pred, Length: 86, dtype: category
Categories (5, int64): [0 < 1 < 2 < 3 < 4]


In [26]:
df['Rating_group'] = pd.qcut(df['pred'], [0, 0.0714, 0.2143, .5, 0.7857, 0.9286, 1.], labels=['D','CC','B','BB','BBB','A'] )
df.to_excel("yahoo_ticker_sample_scraped_grouped_predicted2.xlsx")